In [ ]:

import numpy as np
import librosa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy 
from scipy.stats import kurtosis
from collections import Counter
import pywt


In [ ]:
!pip install pywavelets

In [ ]:
df=pd.read_csv("D:\\aseth_data\\path_set (1)_18_8_2.csv")


In [ ]:
s=df.iloc[139,0]
x,sr=librosa.load(s)
start=int(df.iloc[139,4])*22050
end=int(df.iloc[139,5])*22050
x=x[start:end]

In [ ]:
from scipy.fft import fft
y=fft(x)
print(len(y))
y=np.abs(y)


In [ ]:
def fft_q(yf):    
    N=len(yf)
    T = 1.0 / 800.0
    yf = fft(yf)
    # xf = fftfreq(N, T)[:N//2]
    import matplotlib.pyplot as plt
    xf = np.linspace(0.0, 1.0/(2.0*T), N//2)

    plt.plot(xf,2.0/N * np.abs(yf[:N//2]))
    plt.grid()
    plt.show()

In [ ]:
yf=y
fft_q(yf)
    

In [ ]:
def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy=scipy.stats.entropy(probabilities)
    return entropy
 
def calculate_statistics(list_values):
    n5 = np.nanpercentile(list_values, 5)
    n25 = np.nanpercentile(list_values, 25)
    n75 = np.nanpercentile(list_values, 75)
    n95 = np.nanpercentile(list_values, 95)
    median = np.nanpercentile(list_values, 50)
    mean = np.nanmean(list_values)
    std = np.nanstd(list_values)
    var = np.nanvar(list_values)
    rms = np.nanmean(np.sqrt(np.multiply(list_values,list_values)))
    kurt=kurtosis(list_values)
    skew=scipy.stats.skew(list_values)
    return [n5, n25, n75, n95, median, mean, std, var, rms,kurt,skew]
 
def calculate_crossings(list_values):
    zero_crossing_indices= np.where(np.diff(np.sign(list_values)))[0]
    no_zero_crossings = len(zero_crossing_indices)
    mean_crossing_indices = np.where(np.diff(np.sign(list_values-np.mean(list_values))))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return [no_zero_crossings, no_mean_crossings]
 
def get_features(list_values):
    entropy = calculate_entropy(list_values)
    crossings = calculate_crossings(list_values)
    statistics = calculate_statistics(list_values)
    return [entropy] + crossings + statistics

def codefora():
    k=pd.DataFrame()
    for i in range(0,len(df)):
        feat=[]
        s=df.iloc[i,0]
        print(s)
        x,sr=librosa.load(s)
        wp = pywt.WaveletPacket(data=x, wavelet='sym2', mode='symmetric',maxlevel=3)
        for node in wp.get_level(3, 'freq'):
            print(node.data)
            a=get_features(node.data)
            s=pd.Series(a)
            k=k.append(s,ignore_index=True)
        
    return k
    
def codeforaudio():
    feat=[];
    k=pd.DataFrame()
    for i in range(0,len(df)):
        net=[];s=[]
        s=df.iloc[i,0]
        start=int(df.iloc[i,4])*22050
        end=int(df.iloc[i,5])*22050
        print(s)
        x,sr=librosa.load(s);j=0
        x=x[start:end]
        wp = pywt.WaveletPacket(data=x, wavelet='sym2', mode='symmetric',maxlevel=5)
        for node in wp.get_level(5, 'freq'):
            a=get_features(node.data)
            fft_q(a)
            net.append(a)
        net=np.array(net)
        net=net.flatten()
#         a=get_features(net)
        s=pd.DataFrame(net)    
        s=s.T
#         print(s)
        k=k.append(s, ignore_index=True)
        print(k)
#         a=get_features(k)
#         print(a)
#         feat.append(a)
    return k
    

In [ ]:

f=codeforaudio()

print(len(f_new))
# print(f_new)

In [ ]:
print(f.head())

In [ ]:
label1=df.iloc[:,1].values
label=[]
for i in range(0,len(label1)):
    if label1[i]=='Y':
        label.append(int(1))
    elif label1[i]=='I' or label1[i]=='N' :
        label.append(int(0))
        
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
model=RandomForestClassifier()
x_train,x_test,y_train,y_test=train_test_split(f,label,test_size=0.4)
# x_train=X[:200]
# y_train=label[:200]
# x_test=X[200:]
# y_test=label[200:]
model.fit(x_train,y_train)
print(model.score(x_test,y_test))

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
estimator = RandomForestClassifier()
selector = RFECV(estimator, step=1,cv=5)
selector = selector.fit(x_train, y_train)
print(selector.get_params())
print(selector.score(x_test,y_test))
# print(selector.ranking_array)